# 项目：整理Netflix电影演员评分数据

## 分析目标

此数据分析的目的是，整理不同流派影视作品，**比如喜剧片、动作片、科幻片中，各演员出演作品的平均IMDB评分，从而挖掘出各个流派**中的**高评分作品演员。**

本实战项目的目的在于练习整理数据，从而得到可供下一步分析的数据。

分析目标要牢记在心中，重要数据：流派，演员＆对应评分
高评分作品和演员，不包括导演
第一个数据库主要是作品相关信息
第二个是作品关联到的演员信息
0.先进行合并吗，再分块
1.按照作品流派类型进行分块：喜剧片，动作片，科幻片
2.统计每个演员在这些流派中的评分
3.

## 简介

原始数据集记录了截止至2022年7月美国地区可观看的所有Netflix电视剧及电影数据。数据集包含两个数据表：`titles.csv`和`credits.csv`。

`titles.csv`包含**电影及电视剧相关信息**，包括**影视作品ID**、标题、类型、描述、流派、IMDB（一个国外的在线评分网站）评分，等等。`credits.csv`包含超过7万名出现在Netflix影视作品的**导演及演员信息**，包括名字、**影视作品ID**、人物名、演职员类型（导演/演员）等。

`titles.csv`每列的含义如下：
- id：影视作品ID。
- title：影视作品标题。
- show_type：作品类型，电视节目或电影。
- description：简短描述。
- release_year：发布年份。
- age_certification：适龄认证。
- runtime：每集电视剧或电影的长度。
- genres：流派类型列表。
- production_countries：出品国家列表。
- seasons：如果是电视剧，则是季数。
- imdb_id：IMDB的ID。
- imdb_score：IMDB的评分。
- imdb_votes：IMDB的投票数。
- tmdb_popularity：TMDB的流行度。
- tmdb_score：TMDB的评分。

`credits.csv`每列的含义如下：
- person_ID：演职员ID。
- id：参与的影视作品ID。
- name：姓名。
- character_name：角色姓名。
- role：演职员类型，演员或导演。

## 读取数据

In [1]:
import pandas as pd

In [2]:
original_titles = pd.read_csv("titles.csv")
original_credits = pd.read_csv("credits.csv")

In [3]:
original_credits.head() #是否导入成功

,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR


In [4]:
original_titles.head() #是否导入成功 #没有什么问题

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


### copy原数据集

In [5]:
cleaned_titles = original_titles.copy()
cleaned_credits = original_credits.copy()

## 评估和清理数据

### 数据整齐度

In [6]:
cleaned_titles.sample(10)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
4523,tm949905,Asura Guru,MOVIE,"Shakti, who becomes obsessed with money from c...",2020,NaN,120,"['action', 'crime', 'thriller']",['IN'],NaN,tt8011344,3.2,322.0,NaN,NaN
996,tm158302,Ankhon Dekhi,MOVIE,Bauji resists his daughter's request to let he...,2014,NaN,104,"['drama', 'comedy']",['IN'],NaN,tt3614516,8.0,11330.0,4.013,6.900
3304,ts268799,Equinox,SHOW,Haunted by visions after her sister vanished w...,2020,NaN,46,"['scifi', 'drama']",['DK'],1.0,NaN,NaN,NaN,15.497,7.400
3045,tm441050,The Gentlemen,MOVIE,American expat Mickey Pearson has built a high...,2019,R,113,"['crime', 'action', 'comedy']","['US', 'GB']",NaN,tt8367814,7.8,325385.0,67.919,7.687
2008,ts82218,Hi Score Girl,SHOW,The year is 1991 and 6th grader Yaguchi Haruo ...,2018,TV-PG,27,"['comedy', 'animation', 'action', 'romance']",['JP'],2.0,tt8963328,8.0,2002.0,13.722,8.200
5503,tm1152224,Love Tactics,MOVIE,An ad executive and a fashion designer-blogger...,2022,NaN,97,"['comedy', 'romance']",['TR'],NaN,tt14486678,5.2,3774.0,56.074,6.900
2661,tm449262,Ascharya Fuck It,MOVIE,Desire and greed intertwines the lives of a Bo...,2018,NaN,90,"['thriller', 'drama', 'romance']",['IN'],NaN,tt6531196,5.1,914.0,3.200,5.200
2724,tm299771,Gad Gone Wild,MOVIE,French comic Gad Elmaleh regales a Montreal cr...,2017,NaN,57,"['comedy', 'european']",['FR'],NaN,tt6550440,6.9,408.0,5.985,6.800
2006,ts89130,If I Hadn't Met You,SHOW,"Eduard, a husband and father who loses his fam...",2018,TV-MA,52,"['thriller', 'drama', 'scifi', 'fantasy', 'rom...",['ES'],1.0,tt9817268,7.7,2283.0,4.495,6.900
1356,ts86021,Molang,SHOW,Molang is an affectionate and humorous perspec...,2015,TV-G,3,"['animation', 'comedy', 'family', 'european']",['FR'],3.0,tt6046238,8.3,367.0,9.070,8.000


genes和production国家都有问题，不只是包含一个值，而是多个值

In [7]:
cleaned_titles["genres"][1]#打印一个值看看数据类型对不对，列表才能统计个数，字符串不行

"['drama', 'crime']"

显然这里是字符串，需要转换

In [8]:
cleaned_titles["genres"] = cleaned_titles["genres"].apply(lambda s:eval(s))
cleaned_titles["genres"][1]

['drama', 'crime']

然后进行切分操作

In [9]:
cleaned_titles = cleaned_titles.explode("genres")
cleaned_titles.head(5)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,documentation,['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,crime,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,drama,['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,action,['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300


下面对国家进行操作

In [10]:
cleaned_titles["production_countries"][0]

"['US']"

In [11]:
cleaned_titles["production_countries"] = cleaned_titles["production_countries"].apply(lambda s:eval(s))
cleaned_titles["production_countries"][1]

1    [US]
1    [US]
Name: production_countries, dtype: object

In [12]:
cleaned_titles = cleaned_titles.explode("production_countries")
cleaned_titles.head(5)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,documentation,US,1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,drama,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,action,US,NaN,tt0068473,7.7,107673.0,10.010,7.300


轮到credits了

In [13]:
cleaned_credits.sample(10)

,person_id,id,name,character,role
9359,1550437,tm135451,Asanté Ramone,Neighbor,ACTOR
76025,2205191,tm1150800,Nerina de Jager,NaN,DIRECTOR
43631,805062,tm362737,Nina Iseli,Municipal Council,ACTOR
16256,23713,tm183523,Michael J. Burg,Luxembourg Police Chief,ACTOR
491,225931,tm19608,Faten Hamamah,Amal,ACTOR
31798,61091,tm411311,Martha Howe-Douglas,Female Lip Reader,ACTOR
67911,74801,tm1185374,Jean Harlow,Self (archive footage),ACTOR
48489,696310,tm498803,Post Malone,'Squeeb',ACTOR
69169,102463,tm1038363,Eric Sims,Thomas Jefferson (voice),ACTOR
71541,92350,tm860261,Linda Ko,Aunt Carrie,ACTOR


### 数据干净度

先对titles操作

In [14]:
cleaned_titles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17818 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    17818 non-null  object 
 1   title                 17817 non-null  object 
 2   type                  17818 non-null  object 
 3   description           17790 non-null  object 
 4   release_year          17818 non-null  int64  
 5   age_certification     10889 non-null  object 
 6   runtime               17818 non-null  int64  
 7   genres                17755 non-null  object 
 8   production_countries  17439 non-null  object 
 9   seasons               6224 non-null   float64
 10  imdb_id               17116 non-null  object 
 11  imdb_score            16976 non-null  float64
 12  imdb_votes            16945 non-null  float64
 13  tmdb_popularity       17663 non-null  float64
 14  tmdb_score            17241 non-null  float64
dtypes: float64(5), int64(2), 

In [15]:
#### 格式转换

年份和一个是日期格式

In [16]:
cleaned_titles["release_year"] = pd.to_datetime(cleaned_titles["release_year"],format = "%Y")
cleaned_titles["release_year"]

0      1945-01-01
1      1976-01-01
1      1976-01-01
2      1972-01-01
2      1972-01-01
          ...    
5847   2021-01-01
5848   2021-01-01
5849   2021-01-01
5849   2021-01-01
5849   2021-01-01
Name: release_year, Length: 17818, dtype: datetime64[ns]

credit

In [17]:
cleaned_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77801 entries, 0 to 77800
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  77801 non-null  int64 
 1   id         77801 non-null  object
 2   name       77801 non-null  object
 3   character  68029 non-null  object
 4   role       77801 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.0+ MB


In [18]:
cleaned_credits["person_id"] = cleaned_credits["person_id"].astype(str)
cleaned_credits["person_id"]

0           3748
1          14658
2           7064
3           3739
4          48933
          ...   
77796     736339
77797     399499
77798     373198
77799     378132
77800    1950416
Name: person_id, Length: 77801, dtype: object

#### 处理缺失值

titles

In [19]:
cleaned_titles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17818 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    17818 non-null  object        
 1   title                 17817 non-null  object        
 2   type                  17818 non-null  object        
 3   description           17790 non-null  object        
 4   release_year          17818 non-null  datetime64[ns]
 5   age_certification     10889 non-null  object        
 6   runtime               17818 non-null  int64         
 7   genres                17755 non-null  object        
 8   production_countries  17439 non-null  object        
 9   seasons               6224 non-null   float64       
 10  imdb_id               17116 non-null  object        
 11  imdb_score            16976 non-null  float64       
 12  imdb_votes            16945 non-null  float64       
 13  tmdb_popularity       

流派和评分都有缺失

处理流派

In [20]:
cleaned_titles[cleaned_titles["genres"].isnull()]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
212,tm255589,One Last Shot,MOVIE,"In this low-budget short film, two best buddie...",1998-01-01,NaN,30,NaN,NaN,NaN,NaN,NaN,NaN,1.890,5.2
619,tm341561,Like Twenty Impossibles,MOVIE,Occupied Palestine: A serene landscape now poc...,2003-01-01,NaN,16,NaN,PS,NaN,NaN,NaN,NaN,0.812,6.5
632,ts86241,Le Robe De Mariage Des Cieux,SHOW,It was with much difficulty that Ai Qing was a...,2004-01-01,TV-MA,63,NaN,NaN,1.0,NaN,NaN,NaN,0.600,NaN
636,tm404676,To and from New York,MOVIE,"While covering a story in New York City, a Sea...",2006-01-01,NaN,82,NaN,US,NaN,NaN,NaN,NaN,1.401,5.8
637,tm89054,Osuofia in London 2,MOVIE,Osuofia return to his Nigerian village with a ...,2004-01-01,NaN,72,NaN,XX,NaN,NaN,NaN,NaN,1.091,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5799,tm1040959,The Circle: The Afterparty,MOVIE,Stars of The Circle drop by to discuss Season ...,2021-01-01,NaN,35,NaN,US,NaN,NaN,NaN,NaN,1.882,10.0
5802,ts302434,Plastic Cup Boyz: Laughing My Mask Off!,SHOW,Comedy collective The Plastic Cup Boyz pour ou...,2021-01-01,NaN,33,NaN,NaN,1.0,NaN,NaN,NaN,0.683,NaN
5840,tm1216735,Sun of the Soil,MOVIE,"In 14th-century Mali, an ambitious young royal...",2022-01-01,NaN,26,NaN,NaN,NaN,NaN,NaN,NaN,1.179,7.0
5844,tm1074617,Bling Empire - The Afterparty,MOVIE,"The stars of ""Bling Empire"" discuss the show's...",2021-01-01,NaN,35,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
cleaned_titles.dropna(subset = ["genres"],inplace = True)
cleaned_titles["genres"].isnull().sum()

0

In [22]:
cleaned_titles[cleaned_titles["imdb_score"].isnull()]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945-01-01,TV-MA,51,documentation,US,1.0,NaN,NaN,NaN,0.600,NaN
75,tm132164,Bill Hicks: Sane Man,MOVIE,Sane Man was filmed before Bill recorded ‘Dang...,1989-01-01,R,80,comedy,US,NaN,NaN,NaN,NaN,3.377,7.5
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1991-01-01,TV-G,18,documentation,JP,12.0,NaN,NaN,NaN,7.730,7.8
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1991-01-01,TV-G,18,family,JP,12.0,NaN,NaN,NaN,7.730,7.8
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1991-01-01,TV-G,18,reality,JP,12.0,NaN,NaN,NaN,7.730,7.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5790,tm1094060,My Heroes Were Cowboys,MOVIE,Robin Wiltshire's painful childhood was rescue...,2021-01-01,PG,23,documentation,US,NaN,tt15084326,NaN,NaN,3.145,7.7
5791,tm1047429,Alan Saldaña: Locked Up,MOVIE,"Mexican comedian Alan Saldaña is back, poking ...",2021-01-01,NaN,49,comedy,NaN,NaN,NaN,NaN,NaN,6.670,6.0
5810,tm1225897,Social Man,MOVIE,Two competitive social media Influencers go he...,2021-01-01,NaN,96,comedy,NaN,NaN,tt20198164,NaN,NaN,NaN,NaN
5810,tm1225897,Social Man,MOVIE,Two competitive social media Influencers go he...,2021-01-01,NaN,96,drama,NaN,NaN,tt20198164,NaN,NaN,NaN,NaN


In [23]:
cleaned_titles.dropna(subset = ["imdb_score"],inplace = True)
cleaned_titles["imdb_score"].isnull().sum()

0

credits

In [24]:
cleaned_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77801 entries, 0 to 77800
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  77801 non-null  object
 1   id         77801 non-null  object
 2   name       77801 non-null  object
 3   character  68029 non-null  object
 4   role       77801 non-null  object
dtypes: object(5)
memory usage: 3.0+ MB


#### 处理重复值

由于每一行都是一个作品，每一行不允许重复

titles

In [25]:
cleaned_titles.duplicated().sum()

0

credits

In [26]:
cleaned_credits.duplicated().sum()

0

两个都没有重复数据

#### 处理不一致

titles

流派和国家

In [27]:
cleaned_titles["genres"].value_counts()

genres
drama            3357
comedy           2419
thriller         1446
action           1339
romance          1080
crime            1066
documentation     981
family            769
animation         732
fantasy           727
european          679
scifi             647
horror            438
history           336
music             266
reality           226
war               221
sport             188
western            53
Name: count, dtype: int64

In [28]:
cleaned_titles.query('genres == ""') # 使用 query 方法查找空字符串

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score


这里还需要移除空字符串

In [29]:
cleaned_titles = cleaned_titles.query('genres != ""')
cleaned_titles.query('genres == ""')

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score


国家

In [30]:
with pd.option_context('display.max_rows',None):
    print(cleaned_titles["production_countries"].value_counts())

production_countries
US         5648
IN         1610
GB         1068
JP         1046
FR          720
KR          637
ES          637
CA          608
DE          383
CN          295
MX          264
IT          224
BR          221
AU          217
TR          195
PH          192
AR          150
ID          149
BE          148
TW          133
NG          131
PL          126
ZA          103
NL          102
HK          102
CO           94
EG           93
DK           89
TH           87
SE           81
LB           70
NO           68
AE           52
IE           49
SG           47
XX           43
IL           42
RU           41
CL           35
CH           33
PS           32
BG           31
MY           30
SA           28
AT           28
IS           28
LU           27
NZ           27
PE           26
RO           25
QA           24
CZ           22
JO           19
FI           18
HU           18
UY           15
MA           15
PT           14
KH           10
KW           10
PR            9
PK 

进行替换操作

In [31]:
cleaned_titles["production_countries"] = cleaned_titles["production_countries"].replace({"Lebanon":"LB"})

In [32]:
with pd.option_context('display.max_rows',None):
    print(cleaned_titles["production_countries"].value_counts())

production_countries
US    5648
IN    1610
GB    1068
JP    1046
FR     720
KR     637
ES     637
CA     608
DE     383
CN     295
MX     264
IT     224
BR     221
AU     217
TR     195
PH     192
AR     150
ID     149
BE     148
TW     133
NG     131
PL     126
ZA     103
HK     102
NL     102
CO      94
EG      93
DK      89
TH      87
SE      81
LB      71
NO      68
AE      52
IE      49
SG      47
XX      43
IL      42
RU      41
CL      35
CH      33
PS      32
BG      31
MY      30
AT      28
SA      28
IS      28
LU      27
NZ      27
PE      26
RO      25
QA      24
CZ      22
JO      19
FI      18
HU      18
UY      15
MA      15
PT      14
KW      10
KH      10
PK       9
PR       9
UA       8
MT       8
VN       8
LT       7
IR       7
CD       7
SU       7
TN       7
SN       6
AL       6
KE       6
GH       6
IQ       5
MU       5
CY       5
KN       4
GR       4
IO       4
SY       4
TZ       4
MC       4
GL       3
CM       3
AO       3
BS       3
HR       3
BD       3


处理成功

credits

提示角色可能不一致

In [33]:
cleaned_credits["role"].value_counts()

role
ACTOR       73251
DIRECTOR     4550
Name: count, dtype: int64

从以上输出结果来看，role只有两种可能的值，ACTOR或DIRECTOR，不存在不一致数据。我们可以把这列的类型转换为Category，好处是比字符串类型更节约内存空间，也能表明说值的类型有限。

In [34]:
cleaned_credits["role"] = cleaned_credits["role"].astype("category")
cleaned_credits["role"]

0           ACTOR
1           ACTOR
2           ACTOR
3           ACTOR
4           ACTOR
           ...   
77796       ACTOR
77797       ACTOR
77798       ACTOR
77799       ACTOR
77800    DIRECTOR
Name: role, Length: 77801, dtype: category
Categories (2, object): ['ACTOR', 'DIRECTOR']

#### 处理无效or错误

titles

In [35]:
cleaned_titles.describe()

,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
count,16970,16970.000000,5954.000000,16970.000000,1.694100e+04,16842.000000,16515.000000
mean,2015-11-14 22:42:51.974072064,80.912552,2.455492,6.514207,3.281655e+04,29.396307,6.846933
min,1954-01-01 00:00:00,0.000000,1.000000,1.500000,5.000000e+00,0.600000,1.000000
25%,2015-01-01 00:00:00,45.000000,1.000000,5.800000,7.800000e+02,4.070000,6.200000
50%,2018-01-01 00:00:00,90.000000,2.000000,6.600000,3.508000e+03,10.195000,6.900000
75%,2020-01-01 00:00:00,107.000000,3.000000,7.300000,1.697800e+04,23.639000,7.500000
max,2022-01-01 00:00:00,225.000000,42.000000,9.500000,2.294231e+06,2274.044000,10.000000
std,NaN,39.596172,2.869428,1.131095,1.141492e+05,93.178235,1.078831


没有负数，没有什么问题

Q:
1. 流派类型如果是多个，需不需要将它们分开
2. 国家也是多个，要不要分开
3. 评分如果是nan，直接删除，无用数据。

有无缺失值：
title:缺失没有关系，主要是id来辨识

description：没有关系

score：缺失值，待会检查一下，缺失值对应的id和演员有没有缺失

这两个评分有什么区别

数据类型：无

### 重复数据这里不需要操作吧
貌似：id可以重复，标题这两个需要检查

imdb是nan时候，tmdb不是。重点应该是im，所以需要删掉

这里是不是需要对年份，传唱度，季数都要进行转换

其他好像没有问题

## 整理数据

In [36]:
cleaned_titles

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972-01-01,R,109,drama,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972-01-01,R,109,action,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972-01-01,R,109,thriller,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5846,tm898842,C/O Kaadhal,MOVIE,A heart warming film that explores the concept...,2021-01-01,NaN,134,drama,NaN,NaN,tt11803618,7.7,348.0,NaN,NaN
5847,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021-01-01,NaN,90,comedy,CO,NaN,tt14585902,3.8,68.0,26.005,6.300
5849,ts271048,Mighty Little Bheem: Kite Festival,SHOW,"With winter behind them, Bheem and his townspe...",2021-01-01,NaN,7,family,NaN,1.0,tt13711094,7.8,18.0,2.289,10.000
5849,ts271048,Mighty Little Bheem: Kite Festival,SHOW,"With winter behind them, Bheem and his townspe...",2021-01-01,NaN,7,animation,NaN,1.0,tt13711094,7.8,18.0,2.289,10.000


In [37]:
cleaned_credits

,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR
...,...,...,...,...,...
77796,736339,tm1059008,Adelaida Buscato,María Paz,ACTOR
77797,399499,tm1059008,Luz Stella Luengas,Karen Bayona,ACTOR
77798,373198,tm1059008,Inés Prieto,Fanny,ACTOR
77799,378132,tm1059008,Isabel Gaona,Cacica,ACTOR


#### 连接
确定什么键进行连接，以及连接方式

In [38]:
credits_with_titles = pd.merge(cleaned_titles,cleaned_credits,on = "id",how = "inner")

In [39]:
credits_with_titles.head(10)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,person_id,name,character,role
0,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,3748,Robert De Niro,Travis Bickle,ACTOR
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,14658,Jodie Foster,Iris Steensma,ACTOR
2,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,7064,Albert Brooks,Tom,ACTOR
3,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,3739,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,48933,Cybill Shepherd,Betsy,ACTOR
5,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,32267,Peter Boyle,Wizard,ACTOR
6,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,519612,Leonard Harris,Senator Charles Palantine,ACTOR
7,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,29068,Diahnne Abbott,Concession Girl,ACTOR
8,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,519613,Gino Ardito,Policeman at Rally,ACTOR
9,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,3308,Martin Scorsese,Passenger Watching Silhouette,ACTOR


不关心导演，而只是演员

In [40]:
actors_with_titles = credits_with_titles.query('role == "ACTOR"')#这里列名不需要用引号
actors_with_titles.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,person_id,name,character,role
0,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,3748,Robert De Niro,Travis Bickle,ACTOR
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,14658,Jodie Foster,Iris Steensma,ACTOR
2,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,7064,Albert Brooks,Tom,ACTOR
3,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,3739,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,48933,Cybill Shepherd,Betsy,ACTOR


### step
1. 根据流派，id进行分类
2. 聚合运算 -- 平均值
3. 重置
4. 找到最高分
5. 匹配姓名
6. 对流派进行排序

In [41]:
#根据流派，id进行分类
groupby_genres_and_person_id = actors_with_titles.groupby(["genres","person_id"])#先进行流派，再id
groupby_genres_and_person_id

要进行聚合之后，才能出现dataframe结构

In [42]:
imdb_scoure_groupby_genres_and_person_id = groupby_genres_and_person_id["imdb_score"].mean()
imdb_scoure_groupby_genres_and_person_id

genres   person_id
action   1000         6.866667
         100007       7.000000
         100013       6.400000
         100019       6.500000
         100020       6.500000
                        ...   
western  993735       6.500000
         998673       7.300000
         998674       7.300000
         998675       7.300000
         99940        4.000000
Name: imdb_score, Length: 168881, dtype: float64

In [43]:
#索引重置
imdb_scoure_groupby_genres_and_person_id_df = imdb_scoure_groupby_genres_and_person_id.reset_index()
imdb_scoure_groupby_genres_and_person_id_df

,genres,person_id,imdb_score
0,action,1000,6.866667
1,action,100007,7.000000
2,action,100013,6.400000
3,action,100019,6.500000
4,action,100020,6.500000
...,...,...,...
168876,western,993735,6.500000
168877,western,998673,7.300000
168878,western,998674,7.300000
168879,western,998675,7.300000


要得到这一结果，我们需要再次用genres进行分组，然后提取出imdb_score变量，计算其最大值。
这里不能直接对上述数据进行聚合运算，因为聚合运算是针对全部的筛选列
所以需要重新分组

In [44]:
genres_max_scores = imdb_scoure_groupby_genres_and_person_id_df.groupby('genres')["imdb_score"].max()
genres_max_scores

genres
action           9.3
animation        9.3
comedy           9.2
crime            9.5
documentation    9.1
drama            9.5
european         8.9
family           9.3
fantasy          9.3
history          9.1
horror           9.0
music            8.8
reality          8.9
romance          9.2
scifi            9.3
sport            9.1
thriller         9.5
war              8.8
western          8.9
Name: imdb_score, dtype: float64

In [45]:
#匹配姓名
genres_max_scores_with_person_id = pd.merge(genres_max_scores,imdb_scoure_groupby_genres_and_person_id_df,on = ["genres","imdb_score"])
genres_max_scores_with_person_id

,genres,imdb_score,person_id
0,action,9.3,12790
1,action,9.3,1303
2,action,9.3,21033
3,action,9.3,336830
4,action,9.3,86591
...,...,...,...
131,war,8.8,826547
132,western,8.9,22311
133,western,8.9,28166
134,western,8.9,28180


In [46]:
#获取只有名字还有id的dataframe  从其他dataframe数据集筛选也可以 这里刚刚清洗完的数据最方便
actor_id_with_names = cleaned_credits[["person_id","name"]].drop_duplicates()
actor_id_with_names.head(10)

,person_id,name
0,3748,Robert De Niro
1,14658,Jodie Foster
2,7064,Albert Brooks
3,3739,Harvey Keitel
4,48933,Cybill Shepherd
5,32267,Peter Boyle
6,519612,Leonard Harris
7,29068,Diahnne Abbott
8,519613,Gino Ardito
9,3308,Martin Scorsese


In [47]:
genres_max_scores_with_person_name = pd.merge(genres_max_scores_with_person_id,actor_id_with_names,on = "person_id")
genres_max_scores_with_person_name

,genres,imdb_score,person_id,name
0,action,9.3,12790,Olivia Hack
1,action,9.3,1303,Jessie Flower
2,action,9.3,21033,Zach Tyler
3,action,9.3,336830,André Sogliuzzo
4,action,9.3,86591,Cricket Leigh
...,...,...,...,...
131,war,8.8,826547,Yuto Uemura
132,western,8.9,22311,Koichi Yamadera
133,western,8.9,28166,Megumi Hayashibara
134,western,8.9,28180,Unsho Ishizuka
